In [11]:
!git clone https://github.com/shawinCreates/data-science.git

fatal: destination path 'data-science' already exists and is not an empty directory.


In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
dataset = pd.read_csv('/content/data-science/dataset/thyroid_cancer_risk_data.csv')
dataset.head()

,Patient_ID,Age,Gender,Country,Ethnicity,Family_History,Radiation_Exposure,Iodine_Deficiency,Smoking,Obesity,Diabetes,TSH_Level,T3_Level,T4_Level,Nodule_Size,Thyroid_Cancer_Risk,Diagnosis
0,1,66,Male,Russia,Caucasian,No,Yes,No,No,No,No,9.37,1.67,6.16,1.08,Low,Benign
1,2,29,Male,Germany,Hispanic,No,Yes,No,No,No,No,1.83,1.73,10.54,4.05,Low,Benign
2,3,86,Male,Nigeria,Caucasian,No,No,No,No,No,No,6.26,2.59,10.57,4.61,Low,Benign
3,4,75,Female,India,Asian,No,No,No,No,No,No,4.10,2.62,11.04,2.46,Medium,Benign
4,5,35,Female,Germany,African,Yes,Yes,No,No,No,No,9.10,2.11,10.71,2.11,High,Benign


In [14]:
dataset = dataset.drop(columns=['Patient_ID','Country', 'Ethnicity'])
dataset.head()

,Age,Gender,Family_History,Radiation_Exposure,Iodine_Deficiency,Smoking,Obesity,Diabetes,TSH_Level,T3_Level,T4_Level,Nodule_Size,Thyroid_Cancer_Risk,Diagnosis
0,66,Male,No,Yes,No,No,No,No,9.37,1.67,6.16,1.08,Low,Benign
1,29,Male,No,Yes,No,No,No,No,1.83,1.73,10.54,4.05,Low,Benign
2,86,Male,No,No,No,No,No,No,6.26,2.59,10.57,4.61,Low,Benign
3,75,Female,No,No,No,No,No,No,4.10,2.62,11.04,2.46,Medium,Benign
4,35,Female,Yes,Yes,No,No,No,No,9.10,2.11,10.71,2.11,High,Benign


In [15]:
dataset.isnull().sum().sort_values(ascending=False)

,0
Age,0
Gender,0
Family_History,0
Radiation_Exposure,0
Iodine_Deficiency,0
Smoking,0
Obesity,0
Diabetes,0
TSH_Level,0
T3_Level,0


In [16]:
dataset.describe()

,Age,TSH_Level,T3_Level,T4_Level,Nodule_Size
count,212691.000000,212691.000000,212691.000000,212691.000000,212691.000000
mean,51.918497,5.045102,2.001727,8.246204,2.503403
std,21.632815,2.860264,0.866248,2.164188,1.444631
min,15.000000,0.100000,0.500000,4.500000,0.000000
25%,33.000000,2.570000,1.250000,6.370000,1.250000
50%,52.000000,5.040000,2.000000,8.240000,2.510000
75%,71.000000,7.520000,2.750000,10.120000,3.760000
max,89.000000,10.000000,3.500000,12.000000,5.000000


In [17]:
from sklearn.preprocessing import OrdinalEncoder

In [18]:
binaryColumns = ['Family_History', 'Radiation_Exposure', 'Iodine_Deficiency', 'Smoking', 'Obesity', 'Diabetes']

In [19]:
dataset[binaryColumns] = dataset[binaryColumns].replace({'Yes':1, 'No':0})

dataset['Gender'] = dataset['Gender'].map({'Female':1, 'Male':0})

oe = OrdinalEncoder(categories=[['Low', 'Medium', 'High']])
dataset['Thyroid_Cancer_Risk'] = oe.fit_transform(dataset[['Thyroid_Cancer_Risk']])

/tmp/ipython-input-647240588.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  dataset[binaryColumns] = dataset[binaryColumns].replace({'Yes':1, 'No':0})


In [20]:
dataset.rename(columns={
    'Gender':'Female',
    'Thyroid_Cancer_Risk':'Cancer_Risk'
}, inplace=True)

In [21]:
dataset.head()

,Age,Female,Family_History,Radiation_Exposure,Iodine_Deficiency,Smoking,Obesity,Diabetes,TSH_Level,T3_Level,T4_Level,Nodule_Size,Cancer_Risk,Diagnosis
0,66,0,0,1,0,0,0,0,9.37,1.67,6.16,1.08,0.0,Benign
1,29,0,0,1,0,0,0,0,1.83,1.73,10.54,4.05,0.0,Benign
2,86,0,0,0,0,0,0,0,6.26,2.59,10.57,4.61,0.0,Benign
3,75,1,0,0,0,0,0,0,4.10,2.62,11.04,2.46,1.0,Benign
4,35,1,1,1,0,0,0,0,9.10,2.11,10.71,2.11,2.0,Benign


In [22]:
x = dataset.iloc[:, :-1]
y = dataset['Diagnosis']

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [25]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
rdc = RandomForestClassifier(n_estimators=200, criterion='entropy')
rdc.fit(x_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=200)

In [27]:
rdc.score(x_train, y_train)*100, rdc.score(x_test, y_test)*100

(100.0, 82.53980692076229)

In [28]:
rdc.predict([[29, 0, 0, 1, 0, 0, 0, 0, 1.83, 1.73, 10.54, 4.05, 0.0]])

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


array(['Benign'], dtype=object)

In [29]:
#important feature selection
impFeatures = pd.Series(rdc.feature_importances_, index=x.columns).sort_values(ascending=False)
impFeatures

,0
Cancer_Risk,0.164745
TSH_Level,0.160016
T4_Level,0.158767
Nodule_Size,0.156153
T3_Level,0.151615
Age,0.128398
Family_History,0.014462
Female,0.013815
Obesity,0.012385
Smoking,0.011049


In [30]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [36]:
hyperparameters = {
    "n_estimators": [i for i in range(2, 100)],       # 100 to 999
    "criterion": ['gini', 'entropy', 'log_loss'],       # splitting criteria
    "max_depth": [j for j in range(5, 50)],           # 5 to 99
    "max_features": [k for k in range(3, 13)]          # 3 to 12
}


In [37]:
gs = GridSearchCV(RandomForestClassifier(), param_grid=hyperparameters)
gs.fit(x_train, y_train)

KeyboardInterrupt: 

In [ ]:
gs.best_params_

In [ ]:
gs.best_score_

In [ ]:
rs = RandomizedSearchCV(RandomForestClassifier(), param_distributions=hyperparameters, n_iter=60)
rs.fit(x_train, y_train)

In [ ]:
rs.best_params_

In [ ]:
rs.best_score_